# BASIC IDEA OF THE KERNEL

The data consists of a one dimensional time series x with 600 Mio data points. 

At test time, we will see a time series of length 150'000 to predict the next earthquake.

The idea of this kernel is to randomly sample chunks of length 150'000 from x, 

derive some features and use them to update weights of a recurrent neural net with 150'000 / 1000 = 150 time steps. 

In [19]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook
from scipy import stats

from scipy.signal import hilbert, hann, convolve
from sklearn.linear_model import LinearRegression

#from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

# Fix seeds
from numpy.random import seed
seed(639)
from tensorflow import set_random_seed
set_random_seed(5944)

C:\Users\Higgins\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# Import
#float_data = pd.read_csv("../input/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values
df_train = pd.read_hdf("../input/train.hdf", key='0')

float_data = df_train.values
acoustic_data = df_train['acoustic_data'].values
time_to_failures = df_train['time_to_failure'].values

In [3]:
def classic_sta_lta(x, length_sta, length_lta):
    """
    STA/LTA (short-term average/long-term average)
    For noise-free seismograms, the maximum value of the numerical derivative of the STA/LTA ratio is close to the time of the first arrival.
    (https://en.wikipedia.org/wiki/First_break_picking)
    length_staだけズラした波形の振幅の累積二乗の比率/length_finだけズラした波形の振幅の累積二乗の比率
    """
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    sta[:length_lta - 1] = 0 # Pad zeros
    
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

def max_abs_diff_calssic_sta_lat(x, length_sta, length_lta):
    """
    x:1D-array(m*1)
    return:float
    """
    csl = classic_sta_lta(x, length_sta, length_lta)
    return np.abs(np.diff(csl)).max()

In [4]:
def add_trend_feature(mat, abs_values=False):
    """
    単回帰を用いてデータの平均傾きを出力
    """
    trends = np.zeros([mat.shape[0],1])
    for i in range(mat.shape[0]):
        arr = mat[i]
        idx = np.array(range(len(arr)))
        if abs_values:
            arr = np.abs(arr)
        lr = LinearRegression()
        lr.fit(idx.reshape(-1, 1), arr)
        trends[i] = lr.coef_[0]
    return trends

In [16]:
# Helper function for the data generator. Extracts mean, standard deviation, and quantiles per time step.
# Can easily be extended. Expects a two dimensional array.
def extract_features(z):
    """
    z:2D-array(m*n)
    return:features(m*x)
    """
    return np.c_[z.mean(axis=1), 
                 z.min(axis=1),
                 z.max(axis=1),
                 z.std(axis=1),
                  
                 add_trend_feature(z),
                 add_trend_feature(z, abs_values=True),                  
                 np.abs(z).mean(axis=1),
                 np.abs(z).std(axis=1),
                  
                 #X_out.loc[i, 'mad'] = x_in.mad()
                 np.median(z, axis=1).reshape(-1,1),
                 stats.kurtosis(z, axis=1).reshape(-1,1),
                 stats.skew(z, axis=1).reshape(-1,1),

                 #np.abs(hilbert(x_in)).mean()
                 #convolve(x_in, hann(150), mode='same') / sum(hann(150))).mean()
                ]

In [17]:
# For a given ending position "last_index", we split the last 150'000 values 
# of "x" into 150 pieces of length 1000 each. So n_steps * step_length should equal 150'000.
# From each piece, a set features are extracted. This results in a feature matrix 
# of dimension (150 time steps x features).  
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]

In [19]:


# def extract_features_df(z, col_head=None):
#     """
#     z:2D-array(m*n)
#     return 2D-array(m * num_of_feature)
#     """
#     df_ext = pd.DataFrame(index=range(z.shape[0]))
    
#     df_ext['mean'] = z.mean(axis=1).reshape(-1,1)            
#     df_ext['min'] = z.min(axis=1).reshape(-1,1)             
#     df_ext['max'] = z.max(axis=1).reshape(-1,1)            
#     df_ext['std'] = z.std(axis=1).reshape(-1,1)
#    # df_ext['median'] = np.median(z, axis=1).reshape(-1,1)
#     df_ext['kurtosis'] = stats.kurtosis(z, axis=1).reshape(-1,1)
#     df_ext['skew'] = stats.skew(z, axis=1).reshape(-1,1)

#     for i in range(z.shape[0]): 
#         zs_i = pd.Series(z[i])   
        
#         if z.shape[1] >= 100:
#             #df_ext.loc[i, 'mad_stalat_10-50'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=10, length_lta=50)
#             #df_ext.loc[i, 'mad_stalat_33-66'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=33, length_lta=66)
#             df_ext.loc[i, 'mad_stalat_33-99'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=33, length_lta=99)
#         if z.shape[1] >= 150:
#             #df_ext.loc[i, 'mad_stalat_10-100'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=10, length_lta=100)
#             # df_ext.loc[i, 'mad_stalat_33-123'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=33, length_lta=123)
#             df_ext.loc[i, 'mad_stalat_40-100'] = max_abs_diff_calssic_sta_lat(zs_i, length_sta=40, length_lta=100)
#             #df_ext.loc[i, 'mad_stalat_50-150'] =  max_abs_diff_calssic_sta_lat(zs_i, length_sta=50, length_lta=150)
        
#         # Windowを変化させrolling特徴量を生成
#         for window in [10]: #,30]:
#             if z.shape[1] < window+10:
#                 continue    
#             z_roll_std = zs_i.rolling(window).std().dropna().values
#             z_roll_mean = zs_i.rolling(window).mean().dropna().values   
      
#             df_ext.loc[i, 'ave_roll_std_wd_' + str(window)] = z_roll_std.mean()          
#             df_ext.loc[i, 'std_roll_std_wd_' + str(window)] = z_roll_std.std()
# #             df_ext.loc[i, 'max_roll_std_wd_' + str(window)] = z_roll_std.max()
# #             df_ext.loc[i, 'min_roll_std_wd_' + str(window)] = z_roll_std.min()
#             df_ext.loc[i, 'ave_roll_mean_wd_' + str(window)] = z_roll_mean.mean()
#             df_ext.loc[i, 'std_roll_mean_wd_' + str(window)] = z_roll_mean.std()
# #             df_ext.loc[i, 'max_roll_mean_wd_' + str(window)] = z_roll_mean.max()
# #             df_ext.loc[i, 'min_roll_mean_wd_' + str(window)] = z_roll_mean.min()
    
#     df_ext.dropna(axis=1, inplace=True)
    
#     if col_head is not None:
#         df_ext.columns = [str(col_head) + col for col in df_ext.columns]
    
#     return df_ext

In [14]:
# # For a given ending position "last_index", we split the last 150'000 values 
# # of "x" into 150 pieces of length 1000 each. So n_steps * step_length should equal 150'000.
# # From each piece, a set features are extracted. This results in a feature matrix 
# # of dimension (150 time steps x features).  
# def create_feature_df(x, last_index=None, n_steps=150, step_length=1000):
#     if last_index == None:
#         last_index=len(x)
       
#     assert last_index - n_steps * step_length >= 0

#     # Reshaping and approximate standardization with mean 5 and std 3.
#     #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
#     z = x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1)
    
#     # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
#     # of the last 10 observations. 
# #     return np.c_[extract_features(temp),
# #                  extract_features(temp[:, -step_length // 10:]),
# #                  extract_features(temp[:, -step_length // 100:])]
#     return pd.concat([extract_features_df(z, 'all_'), 
#                       extract_features_df(z[:, -step_length // 10:], 'last100_'), 
#                       extract_features_df(z[:, -step_length // 100:], 'last10_')], axis=1)

In [20]:
# Query "create_X" to figure out the number of features
#df_feat_tr = create_feature_df(acoustic_data[0:150000])
#n_features = df_feat_tr.shape[1]
data_tr = create_X(float_data[0:150000])
n_features = data_tr.shape[1]
print("Our RNN is based on %i features"% n_features)

Our RNN is based on 33 features


In [27]:
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
        
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[row - 1, 1]
        yield samples, targets

In [28]:
# # The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# # the "time_to_failure" serves as target, while the features are created by the function "create_X".
# def generator_parallel(data, min_index=0, max_index=None, batch_size=10000, n_steps=150, step_length=1000):
#     if max_index is None:
#         max_index = len(data) - 1
        
#     #count = 1
#     while True:
#         #print('Generate:', count)
#         #count += 1
        
#         # Pick indices of ending positions
#         rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
#         def process(last_index):
#             # Reshaping and approximate standardization with mean 5 and std 3.
#             z = data[(last_index - n_steps * step_length):last_index,0].reshape(n_steps, -1)

#             # Extracts features of sequences of full length 1000, of the last 100 & 10 observations. 
#             df_feat_all = extract_features_df(z, 'all_')
#             df_feat_last100 = extract_features_df(z[:, -step_length // 10:], 'last100_')
#             df_feat_last10 = extract_features_df(z[:, -step_length // 100:], 'last10_')
#             features = np.c_[df_feat_all.values, df_feat_last100.values, df_feat_last10.values]
#             # create_feature_df(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length).values
#             return features

#         samples = np.array(Parallel(n_jobs=-1)([delayed(process)(r) for r in rows]))
#         targets = np.array([data[r - 1, 1] for r in rows])
        
#         yield samples, targets

In [30]:
# # The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# # the "time_to_failure" serves as target, while the features are created by the function "create_X".
# def generate_data(data, size, min_index=0, max_index=None,  n_steps=150, step_length=1000):
#     if max_index is None:
#         max_index = len(data) - 1
        
#     # Pick indices of ending positions
#     rows = np.random.randint(min_index + n_steps * step_length, max_index, size=size)

#     # Initialize feature matrices and targets
#     samples = np.zeros((size, n_steps, n_features))
#     targets = np.zeros(size, )
    
#     def process(last_index):
#         # Reshaping and approximate standardization with mean 5 and std 3.
#         z = data[(last_index - n_steps * step_length):last_index,0].reshape(n_steps, -1)

#         # Extracts features of sequences of full length 1000, of the last 100 & 10 observations. 
#         df_feat_all = extract_features_df(z, 'all_')
#         df_feat_last100 = extract_features_df(z[:, -step_length // 10:], 'last100_')
#         df_feat_last10 = extract_features_df(z[:, -step_length // 100:], 'last10_')
#         features = np.c_[df_feat_all.values, df_feat_last100.values, df_feat_last10.values]

#         return features
    
#     # 100サンプルごとに更新
#     update_freq = 100
#     for i in tqdm_notebook(range(size//update_freq)):
#         j_min = i*update_freq 
#         j_max = np.min([size, (i+1)*update_freq])
#         rows_i = rows[j_min:j_max]
#         samples[j_min:j_max] = np.array(Parallel(n_jobs=-1)([delayed(process)(r) for r in rows_i]))
#         targets[j_min:j_max] = np.array([data[r - 1, 1] for r in rows_i])
        
#     return samples, targets

In [31]:
# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877

In [32]:
# Initialize generators
batch_size = 32

# #train_gen = generator(data=df_train.values, batch_size=batch_size) # Use this for better score
train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(data=df_train.values, batch_size=batch_size, max_index=second_earthquake)

In [114]:
# x_train, y_train = generate_data(float_data, size=1000, min_index=second_earthquake + 1)
# x_valid, y_valid = generate_data(float_data, size=100, max_index=second_earthquake)

In [12]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
from keras import activations, regularizers, initializers, constraints
from keras.engine import Layer, InputSpec

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
# This is NN LSTM Model creation
def model_lstm(input_shape, feat_shape):
    inp = Input(shape=(input_shape[1], input_shape[2],))
    feat = Input(shape=(feat_shape[1],))

    bi_lstm = Bidirectional(CuDNNLSTM(128, return_sequences=True), merge_mode='concat')(inp)
    bi_gru = Bidirectional(CuDNNGRU(64, return_sequences=True), merge_mode='concat')(bi_lstm)
    
    attention = Attention(input_shape[1])(bi_gru)
    
    x = concatenate([attention, feat], axis=1)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[inp, feat], outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model

In [33]:
# Define model
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU, Bidirectional
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint, TensorBoard

cb = [
    ModelCheckpoint("model.hdf5", save_best_only=True, period=3),
    #TensorBoard(log_dir="tflog/", histogram_freq=1)
]

model = Sequential()
model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnngru_3 (CuDNNGRU)       (None, 48)                11952     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                490       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 12,453
Trainable params: 12,453
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile and fit model
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=128,
                              epochs=50,
                              verbose=1,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

# history = model.fit(x_train, y_train,
#                     batch_size=1000,
#                     epochs=1000,
#                     verbose=2,
#                     callbacks=cb,
#                     validation_data=(x_valid, y_valid),
#                     #validation_steps=200
#                    )

Epoch 1/50


UnknownError: Fail to find the dnn implementation.
	 [[{{node cu_dnngru_3/CudnnRNN}}]]
	 [[{{node loss_3/mul}}]]

In [ ]:
# Visualize accuracies
import matplotlib.pyplot as plt

def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

In [ ]:
# Load submission file
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

# Save
submission.to_csv('submission.csv')